# Concurrent Programming (cntd.)

## threading 
 * `Thread` is the main class 
 `Lock, RLock, Semaphore, Condition` are basic synchronization classes
 
In contrast to `multiprocessing`, threads work in the same memory environment. All global variables and parameters passed by reference to thread function are shared.
 
Methods and data types are conveniently similar.
 
Following is the ping/pong example with threading. Note the value of the counter. Also note the race condition in print(). Threads even share the same buffer for the file descriptors.

In [7]:
import threading as th

In [5]:
counter = 10
def ping(name,memut,othmut):
    global counter
    for i in range(0,5):
        # wait until my turn
        memut.acquire()
        print(name,counter)
        # tell other end it is its turn
        othmut.release()
        counter += 1

        
imut,omut = th.Lock(), th.Lock()

pip = th.Thread(target=ping, args=("ping",imut,omut))
pop = th.Thread(target=ping, args=("pong",omut,imut))

# make sure only one (ping enters first)
omut.acquire()
pip.start()
pop.start()
pip.join()
pop.join()
omut.release()
print("in main thread: {}".format(counter))
    

ping 10
pong 11
ping 12
pong 13
ping 14
pong 15
ping 16
pong 17
ping 18
pong 19
in main thread: 20


## A Race Condition Example

### Global Interpreter Lock (GIL)
All python code execute in a critical region defined by GIL to protect internal interpreter data structures

Python has an internal scheduler that preempts executing thread and give CPU to another. Race condition is still an issue.

GIL does not save you from race conditions!

When thread waits for I/O or executes native C  code (numpy, or other C based libraries), GIL is released.


## What about multiprocessing?

There are multiple interpreters, Python process. GIL is per process. multiprocessing creates multiple process. They work in parallel without any issue.

In [30]:
''' Race condition of threads.
    uncomment mut to use a mutex. 
    You need long executing threads with
    multi-line critical regions to observe it.'''
mut = th.Lock()

def f(lst):
    for i in range(0,50000):
        # the following is the critical region
        #with mut:
            #mut.acquire()
            #tmp = lst[0]
            #tmp += 1
            #lst[1] # exception
            #return # return
            #lst[0] = tmp
            lst[0] += 1
            #mut.release()
    
     
arr = [0]

threads = [th.Thread(target=f, args=(arr,)) for i in range(0,100)]

print(arr)
for t in threads:
    t.start()

for t in threads:
    t.join()
    
print(arr)



[0]
[4950000]


In [8]:
import time
import multiprocessing as mp

counter = mp.Value('i',10)

def ping(name,memut,othmut):
    global counter
    for i in range(0,5):
        # wait until my turn
        memut.acquire()
        with counter:
            counter.value += 1
        print(name,counter.value)
        # tell other end it is its turn
        othmut.release()
    time.sleep(30)
        
        
imut,omut = mp.Lock(), mp.Lock()

pip = mp.Process(target=ping, args=("ping",imut,omut))
pop = mp.Process(target=ping, args=("pong",omut,imut))

# make sure only one (ping enters first)
omut.acquire()
pip.start()
pop.start()
pip.join()
pop.join()
omut.release()
print("in main thread: {}".format(counter.value))

ping 11
pong 12
ping 13
pong 14
ping 15
pong 16
ping 17
pong 18
ping 19
pong 20
in main thread: 20


## Shared Variables in `multiprocessing`

Shared globals and heap is provided by its nature in `threading`. In multiprocessing standard variables except synchronization primitives are isolated in each process. In order to share values `Value` and `Array` classes provide shared variables with atomic access that multiple process can use.

  * `Value` constructor needs a type specifier argument either a `ctypes` type or a character representing it:
     > `'i'` equivalent to `c_int`
     >
     > `'c'` equivalent to `c_char`
     >
     > `'f'` equivalent to `c_float`
     >
     > `'d'` equivalent to `c_double`
     >
     
     Depending on the parameter a shared variable with given type is created.
  * Use `value` member of the `Value` objects to get/update their value.
  * `Array` creates homogeneous list of values. Constructor gets a type specifier and either a size or an iterator:
     > `Array('d',10)`
     >
     > `Array('i',(0,0,1,0,0,1,0,0,1))`

In [9]:
import multiprocessing as mp
v = mp.Value('d')
v.acquire()
v.value = v.value + 10
v.release()
with v:
    v.value += 1
print(v)

a = mp.Array('d',100)
# double a[100];
print(a)

<Synchronized wrapper for c_double(11.0)>
<SynchronizedArray wrapper for <multiprocessing.sharedctypes.c_double_Array_100 object at 0x7f915c484830>>


In [39]:
from multiprocessing import Value,Array

a = Value('i',3)   # a shared integer
a.value += 1
print(a)

v = Array('d',10)  # a shared array of doubles
for i in range(len(v)):
    v[i] = i                    # set and get like a list
v[:] = [0,3,4,5,2,3,4,5,1,9]    # can be updated with slices
print(list(v))                  # is iteratable

m = Array('c', b'Hello World')   # a character buffer (size of Hello World)
m.value = b'hello world'   # it will overflow with larger strings
m[5] = b'X'
print(m.value)

<Synchronized wrapper for c_int(4)>
[0.0, 3.0, 4.0, 5.0, 2.0, 3.0, 4.0, 5.0, 1.0, 9.0]
b'helloXworld'


In [40]:
import time

counter = mp.Value('i',10)

def hello(name):
    global counter
    for i in range(0,5):
        counter.value += 1
        time.sleep(0.5)
        print("hello " + name, counter)
            
p = mp.Process(target=hello, args=("world",))

q = mp.Process(target=hello, args=("myself",))

p.start() 
q.start()
# p, q and main process are concurrent here

p.join() # wait for p to complete
q.join() # wait for q to complete
print('counter is :',counter.value)
# back to single process again

hello world <Synchronized wrapper for c_int(12)>
hello myself <Synchronized wrapper for c_int(13)>
hello world <Synchronized wrapper for c_int(14)>
hello myself <Synchronized wrapper for c_int(15)>
hello world <Synchronized wrapper for c_int(16)>
hello myself <Synchronized wrapper for c_int(17)>
hello world <Synchronized wrapper for c_int(18)>
hello myself <Synchronized wrapper for c_int(19)>
hello world <Synchronized wrapper for c_int(20)>hello myself
 <Synchronized wrapper for c_int(20)>
counter is : 20


## Lock versus RLock

`RLock` is called a recursive lock. Basically it is a lock only holder can release. In `Lock`, any thread can release a lock like in our ping/pong example above. In recursive lock, only thread/process that acquired it can release it.
``` 
```

In [ ]:
a, b = th.Lock(), th.RLock() 

def f():
    a.acquire()
    # This will give error
    b.release()

f()


## Using `with` with synchronization

* `with` provides a semantic context for a data type derived from context manager
* `with` block calls `_enter_` and `_exit_` methods of the context.
* This way, statement blocks that are valid in the context can be defined as:
```python
with open("myfile","r") as fd:
    line = fd.readline()
    while line:
        print(line, end='')
        line = fd.readline()
print('finished')
```
* context manager for a file object automatically closes the file when block is terminated.
* similarly database connections can use `with`

```python
with sqlite3.connect('myfile.sqlite3') as cur:
    cur.execute('....')
    ...
    ....
    
# here connection is closed
```
 
Synchronization classes `Lock, RLock, Semaphore, Condition` can be used similarly:
```python
a=Lock()
with a:
    y = x + 1
    x = y
``` 

Is equivalent to:
```python
a=Lock()
try:
    a.acquire()
    y = x + 1
    x = y
finally:
    a.release()
``` 

Exceptions are handled properly. This form emphasizes the critical regions better.


# Barriers
 * A barrier is a meeting point for all threads. For example each thread makes some initializations and when and only when all of them is ready, the algorithm can start.
 

In [13]:
import random
import time

class MyBarrier:
    def __init__(self, number = 2):
        self.number = number
        self.mutex = th.RLock()
        self.go = th.Lock()
        self.go.acquire()
        self.current = 0
        
    def arrived(self):
        self.mutex.acquire()
        self.current += 1
        print("arrived {}th".format(self.current))
        if self.current == self.number:
            self.current -= 1
            self.mutex.release()
            self.go.release()
        else:
            self.mutex.release()
            self.go.acquire()
            self.mutex.acquire()
            self.current -= 1
            if self.current > 0:
                self.go.release()
            self.mutex.release()
        #print("left {}".format(self.current))
        
def barriertest(barrier):
    time.sleep(1 + random.random()*3)
    barrier.arrived()
    print("entered")
    time.sleep(1 + random.random()*5)
    print("completed")
    # use same barrier for completion
    #barrier.arrived()
    
bar = MyBarrier(6)

t1 = th.Thread(target=barriertest, args=(bar,))
t2 = th.Thread(target=barriertest, args=(bar,))
t3 = th.Thread(target=barriertest, args=(bar,))
t4 = th.Thread(target=barriertest, args=(bar,))
t5 = th.Thread(target=barriertest, args=(bar,))
t6 = th.Thread(target=barriertest, args=(bar,))

t1.start()
t2.start()
t3.start()
t4.start()
t5.start()
t6.start()

t1.join()
t2.join()
t3.join()
t4.join()
t5.join()
t6.join()
print("all terminated")        

arrived 1th
arrived 2th
arrived 3th
arrived 4th
arrived 5th
arrived 6th
entered
entered
entered
entered
entered
entered
completed
completed
completed
completed
completed
completed
all terminated
